In [3]:
horse_ids = pd.read_pickle("horseid.pickle")

In [2]:
import pandas as pd
import time
from tqdm.notebook import tqdm

def scrape_peds(horse_id_list, pre_peds={}):
    peds = pre_peds
    for horse_id in tqdm(horse_id_list):
        if horse_id in peds.keys():
            continue
        try:
            url = "https://db.netkeiba.com/horse/ped/" + horse_id
            df = pd.read_html(url)[0]
            
            generations = {}
            for i in reversed(range(5)):
                generations[i] = df[i]
                df.drop([i], axis=1, inplace=True)
                df = df.drop_duplicates()
            
            ped = pd.concat([generations[i] for i in range(5)]).rename(horse_id)
            peds[horse_id] = ped.reset_index(drop=True)
            time.sleep(1)
        except IndexError:
            continue
        except:
            break
    return peds

In [3]:
horses = pd.read_csv('./csv_new2/horse_id.csv')
targets = horses['horse_id'].astype(str)
peds = scrape_peds(targets)

  0%|          | 0/52 [00:00<?, ?it/s]

In [4]:
peds = pd.concat([peds[horse_id] for horse_id in peds], axis=1).T
peds = peds.add_prefix('peds_')
peds.to_pickle('peds4.pickle')
p = pd.read_pickle('peds4.pickle')
p.to_csv('peds_1.csv')